In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator



In [2]:
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import math
from qiskit.quantum_info import Pauli, PauliList
import numpy as np
import pandas as pd
import copy
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import time

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.cluster import SpectralClustering
from sklearn.metrics import normalized_mutual_info_score
from qiskit import BasicAer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.utils.algorithm_globals import algorithm_globals
from qiskit.optimization.applications.ising import max_cut
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
#from qiskit_machine_learning.algorithms import QSVC
#from qiskit_machine_learning.kernels import QuantumKernel

from qiskit_optimization.applications import Maxcut, Tsp
from qiskit_optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer, GurobiOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.problems.variable import VarType
from qiskit_optimization.converters.quadratic_program_to_qubo import QuadraticProgramToQubo
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp
from qiskit.aqua import *

from qiskit_optimization.applications.graph_optimization_application import GraphOptimizationApplication
from qiskit.providers.ibmq.runtime import UserMessenger, ProgramBackend
from qiskit.algorithms.optimizers import SPSA, COBYLA,SLSQP
from qiskit.optimization.applications.ising import graph_partition
from qiskit.optimization.applications.ising.common import random_graph, sample_most_likely
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.operators import *

from qiskit_optimization.runtime import QAOAProgram, VQEProgram
from qiskit.providers.ibmq.runtime import UserMessenger, ProgramBackend
import random
# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging

/opt/conda/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/opt/conda/lib/python3.8/site-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


In [3]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit import Aer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
import numpy as np

In [136]:
val = [5,6,7,8,9]
wt = [4,5,6,7,8]
W = 18

In [137]:
def dp(W, wt, val, n):
    k = [[0 for x in range(W + 1)] for x in range(n + 1)]
    for i in range(n + 1):
        for w in range(W + 1):
            if i == 0 or w == 0:
                k[i][w] = 0
            elif wt[i-1] <= w:
                k[i][w] = max(val[i-1] + k[i-1][w-wt[i-1]], k[i-1][w])
            else:
                k[i][w] = k[i-1][w]
                
    picks=[0 for x in range(n)]
    volume=W
    for i in range(n,-1,-1):
        if (k[i][volume]>k[i-1][volume]):
            picks[i-1]=1
            volume -= wt[i-1]
    return k[n][W],picks

n = len(val)
print("optimal value:", dp(W, wt, val, n)[0])
print('\n index of the chosen items:')
for i in range(n): 
    if dp(W, wt, val, n)[1][i]: 
        print(i,end=' ')

optimal value: 21

 index of the chosen items:
1 2 3 

In [4]:
# import packages necessary for application classes.
from qiskit_optimization.applications import Knapsack

In [139]:
def knapsack_quadratic_program():
    # Put values, weights and max_weight parameter for the Knapsack()
    
    ##############################
    # Provide your code here
    
    # prob = Knapsack('Insert parameters here')
    prob = Knapsack(values = val, weights = wt, max_weight=W)
    #
    ##############################
    
    # to_quadratic_program generates a corresponding QuadraticProgram of the instance of the knapsack problem.
    kqp = prob.to_quadratic_program()
    return prob, kqp

prob,quadratic_program=knapsack_quadratic_program()
quadratic_program

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Knapsack

Maximize
 obj: 5 x_0 + 6 x_1 + 7 x_2 + 8 x_3 + 9 x_4
Subject To
 c0: 4 x_0 + 5 x_1 + 6 x_2 + 7 x_3 + 8 x_4 <= 18

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4
End

In [140]:
# Numpy Eigensolver
meo = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
result = meo.solve(quadratic_program)
print('result:\n', result)
print('\n index of the chosen items:', prob.interpret(result)) 

result:
 optimal function value: 21.0
optimal value: [0. 1. 1. 1. 0.]
status: SUCCESS

 index of the chosen items: [1, 2, 3]


In [141]:
seed = 123
algorithm_globals.random_seed = seed
qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=seed, seed_transpiler=seed)

meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
result = meo.solve(quadratic_program)
print('result:\n', result)
print('\n index of the chosen items:', prob.interpret(result)) 

result:
 optimal function value: 21.0
optimal value: [0. 1. 1. 1. 0.]
status: SUCCESS

 index of the chosen items: [1, 2, 3]


In [179]:
L1 = [5,3,3,6,8,7,1]
L2 = [8,4,5,12,13,11,2]
C2 = [3,2,3,2,4,3,3]
C_max = 12

In [143]:
test=([[5,3,3,6,8,7,1], [8,4,5,12,13,11,2], [3,2,3,2,4,3,3] ,12],[[5,2,3,5,8,4,1], [8,4,10,12,13,13,2], [3,2,5,2,6,4,3] ,10],[[5,3,5,6,4,6,1], [8,4,6,15,14,11,12], [3,3,3,2,4,3,3] ,15],[[4,3,3,6,5,7,3], [11,4,15,11,13,11,14], [3,1,4,2,6,3,2] ,9],[[5,4,3,4,8,7,1], [8,8,5,11,13,11,2], [3,3,1,2,4,4,3] ,11])

In [144]:
j=0
test[j][0]

[5, 3, 3, 6, 8, 7, 1]

In [145]:
len(test)

5

In [146]:
np.array(test[0][1])-np.array(test[0][0])

array([3, 1, 2, 6, 5, 4, 1])

In [5]:
L11 = [5,3,3,6,8,7,1]
L12 = [8,4,5,12,13,11,2]
C12 = [3,2,3,2,4,3,3]
C1_max = 12

L21 = [4,2,2,5,9,8,1]
L22 = [8,4,5,12,13,11,2]
C22 = [3,4,5,4,8,6,6]
C2_max = 15

L31 = [4,2,1,3,6,5,1]
L32 = [8,4,4,11,15,14,6]
C32 = [4,3,5,1,2,5,7]
C3_max = 18

L41 = [2,3,4,5,8,7,3]
L42 = [8,8,9,12,14,13,9]
C42 = [4,5,2,3,4,2,3]
C4_max = 19

L51 = [3,5,4,6,1,3,2]
L52 = [7,10,6,9,7,10,8]
C52 = [3,2,3,2,4,3,3]
C5_max = 14

L61 = [5,3,5,5,9,8,2]
L62 = [8,4,7,11,12,11,8]
C62 = [3,2,4,5,6,4,2]
C6_max = 10


In [6]:
def knapsack_argument(L11, L12, C12, C1_max):
      
    ##############################
    # Provide your code here
    values = list(map(lambda x, y: x - y, L12, L11))
    weights = C12
    max_weight = C1_max 
    #
    ##############################
    return values, weights, max_weight
    
values, weights, max_weight = knapsack_argument(L11, L12, C12, C1_max)
print(values, weights, max_weight)
prob = Knapsack(values = values, weights = weights, max_weight = max_weight)
qp = prob.to_quadratic_program()
print(qp)
# QAOA
seed = 123
algorithm_globals.random_seed = seed
qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=seed, seed_transpiler=seed)

meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
result = meo.solve(qp)
print('result:', result.x)

item = np.array(result.x)
revenue=0
for i in range(len(item)):
    if item[i]==0:
        revenue+=L11[i]
    else:
        revenue+=L12[i]

print('total revenue:', revenue)


def knapsack_argument(L21, L22, C22, C2_max):
      
    ##############################
    # Provide your code here
    values = list(map(lambda x, y: x - y, L22, L21))
    weights = C22
    max_weight = C2_max 
    #
    ##############################
    return values, weights, max_weight
    
values, weights, max_weight = knapsack_argument(L21, L22, C22, C2_max)
print(values, weights, max_weight)
prob = Knapsack(values = values, weights = weights, max_weight = max_weight)
qp = prob.to_quadratic_program()
print(qp)
# QAOA
seed = 123
algorithm_globals.random_seed = seed
qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=seed, seed_transpiler=seed)

meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
result = meo.solve(qp)
print('result:', result.x)

item = np.array(result.x)
revenue=0
for i in range(len(item)):
    if item[i]==0:
        revenue+=L21[i]
    else:
        revenue+=L22[i]

print('total revenue:', revenue)

def knapsack_argument(L31, L32, C32, C3_max):
      
    ##############################
    # Provide your code here
    values = list(map(lambda x, y: x - y, L22, L21))
    weights = C32
    max_weight = C3_max 
    #
    ##############################
    return values, weights, max_weight
    
values, weights, max_weight = knapsack_argument(L31, L32, C32, C3_max)
print(values, weights, max_weight)
prob = Knapsack(values = values, weights = weights, max_weight = max_weight)
qp = prob.to_quadratic_program()
print(qp)
# QAOA
seed = 123
algorithm_globals.random_seed = seed
qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=seed, seed_transpiler=seed)

meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
result = meo.solve(qp)
print('result:', result.x)

item = np.array(result.x)
revenue=0
for i in range(len(item)):
    if item[i]==0:
        revenue+=L31[i]
    else:
        revenue+=L32[i]

print('total revenue:', revenue)

def knapsack_argument(L41, L42, C42, C4_max):
      
    ##############################
    # Provide your code here
    values = list(map(lambda x, y: x - y, L42, L41))
    weights = C42
    max_weight = C4_max 
    #
    ##############################
    return values, weights, max_weight
    
values, weights, max_weight = knapsack_argument(L41, L42, C42, C4_max)
print(values, weights, max_weight)
prob = Knapsack(values = values, weights = weights, max_weight = max_weight)
qp = prob.to_quadratic_program()
print(qp)
# QAOA
seed = 123
algorithm_globals.random_seed = seed
qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=seed, seed_transpiler=seed)

meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
result = meo.solve(qp)
print('result:', result.x)

item = np.array(result.x)
revenue=0
for i in range(len(item)):
    if item[i]==0:
        revenue+=L41[i]
    else:
        revenue+=L42[i]

print('total revenue:', revenue)

def knapsack_argument(L51, L52, C52, C5_max):
      
    ##############################
    # Provide your code here
    values = list(map(lambda x, y: x - y, L22, L21))
    weights = C52
    max_weight = C5_max 
    #
    ##############################
    return values, weights, max_weight
    
values, weights, max_weight = knapsack_argument(L51, L52, C52, C5_max)
print(values, weights, max_weight)
prob = Knapsack(values = values, weights = weights, max_weight = max_weight)
qp = prob.to_quadratic_program()
print(qp)
# QAOA
seed = 123
algorithm_globals.random_seed = seed
qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=seed, seed_transpiler=seed)

meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
result = meo.solve(qp)
print('result:', result.x)

item = np.array(result.x)
revenue=0
for i in range(len(item)):
    if item[i]==0:
        revenue+=L51[i]
    else:
        revenue+=L52[i]

print('total revenue:', revenue)

def knapsack_argument(L61, L62, C62, C6_max):
      
    ##############################
    # Provide your code here
    values = list(map(lambda x, y: x - y, L62, L61))
    weights = C62
    max_weight = C6_max 
    #
    ##############################
    return values, weights, max_weight
    
values, weights, max_weight = knapsack_argument(L61, L62, C62, C6_max)
print(values, weights, max_weight)
prob = Knapsack(values = values, weights = weights, max_weight = max_weight)
qp = prob.to_quadratic_program()
print(qp)
# QAOA
seed = 123
algorithm_globals.random_seed = seed
qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=seed, seed_transpiler=seed)

meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
result = meo.solve(qp)
print('result:', result.x)

item = np.array(result.x)
revenue=0
for i in range(len(item)):
    if item[i]==0:
        revenue+=L61[i]
    else:
        revenue+=L62[i]

print('total revenue:', revenue)


[3, 1, 2, 6, 5, 4, 1] [3, 2, 3, 2, 4, 3, 3] 12
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Knapsack

Maximize
 obj: 3 x_0 + x_1 + 2 x_2 + 6 x_3 + 5 x_4 + 4 x_5 + x_6
Subject To
 c0: 3 x_0 + 2 x_1 + 3 x_2 + 2 x_3 + 4 x_4 + 3 x_5 + 3 x_6 <= 12

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5 x_6
End

result: [1. 0. 0. 1. 1. 1. 0.]
total revenue: 51
[4, 2, 3, 7, 4, 3, 1] [3, 4, 5, 4, 8, 6, 6] 15
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Knapsack

Maximize
 obj: 4 x_0 + 2 x_1 + 3 x_2 + 7 x_3 + 4 x_4 + 3 x_5 + x_6
Subject To
 c0: 3 x_0 + 4 x_1 + 5 x_2 + 4 x_3 + 8 x_4 + 6 x_5 + 6 x_6 <= 15

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5 x_6
End

result: [1. 0. 0. 1. 1. 0. 0.]
total revenue: 46
[4, 2, 3, 7, 4, 3, 1] [4, 3, 5, 

In [68]:
# QAOA
seed = 123
algorithm_globals.random_seed = seed
qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=seed, seed_transpiler=seed)

meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
result = meo.solve(qp)
print('result:', result.x)

item = np.array(result.x)
revenue=0
for i in range(len(item)):
    if item[i]==0:
        revenue+=L1[i]
    else:
        revenue+=L2[i]

print('total revenue:', revenue)

result: [1. 1. 1. 1. 0. 1. 1.]
total revenue: 51
